In [1]:
# # in google colab uncomment this

# import os

# os.system('apt-get install -y xvfb')
# os.system('wget https://raw.githubusercontent.com/yandexdataschool/Practical_DL/fall18/xvfb -O ../xvfb')
# os.system('apt-get install -y python-opengl ffmpeg')
# os.system('pip install pyglet==1.2.4')
# os.system('pip install gym')

# prefix = 'https://raw.githubusercontent.com/yandexdataschool/Practical_RL/master/week09_policy_II/'

# os.system('wget ' + prefix + 'runners.py')
# os.system('wget ' + prefix + 'mujoco_wrappers.py')

# print('setup complete')

# XVFB will be launched if you run on a server
import os
if type(os.environ.get("DISPLAY")) is not str or len(os.environ.get("DISPLAY")) == 0:
    !bash ../xvfb start
    %env DISPLAY = : 1

# Implementing Proximal Policy Optimization 


In this notebook you will be implementing Proximal Policy Optimization algorithm, 
scaled up version of which was used to train [OpenAI Five](https://openai.com/blog/openai-five/) 
to [win](https://openai.com/blog/how-to-train-your-openai-five/) against the
world champions in Dota 2.
You will be solving a continuous control environment on which it may be easier and faster 
to train an agent, however note that PPO here may not be the best algorithm as, for example,
Deep Deterministic Policy Gradient and Soft Actor Critic may be more suited 
for continuous control environments. ~~To run the environment you will need to install 
[pybullet-gym](https://github.com/benelot/pybullet-gym) which unlike MuJoCo 
does not require you to have a license.~~

To install the library:

#### Installed pybullet  instead of pybulletgym
https://github.com/bulletphysics/bullet3
 

The overall structure of the code is similar to the one in the A2C optional homework, but don't worry if you haven't done it, it should be relatively easy to figure it out. 
First, we will create an instance of the environment. 
We will normalize the observations and rewards, but before that you will need a wrapper that will 
write summaries, mainly, the total reward during an episode. You can either use one for `TensorFlow` 
implemented in `atari_wrappers.py` file from the optional A2C homework, or implement your own. 

In [29]:
import gym 
import pybullet_envs


env = gym.make("HalfCheetahBulletEnv-v0")
print("observation space: ", env.observation_space,
      "\nobservations:", env.reset())
print("action space: ", env.action_space, 
      "\naction_sample: ", env.action_space.sample())

observation space:  Box(26,) 
observations: [ 0.          0.          1.          0.          0.          0.
  0.         -0.         -0.38433948  0.         -0.1040039   0.
 -0.17367972  0.          0.33260956  0.          0.11239259  0.
  1.2187908   0.          0.          0.          0.          0.
  0.          0.        ]
action space:  Box(6,) 
action_sample:  [0.57659733 0.26563177 0.5228279  0.74690473 0.75159585 0.23399161]


In [30]:
import torch
import numpy as np
from collections import deque
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

def torch_steps():
    #https://discuss.pytorch.org/t/current-step-from-optimizer/19370
    try:
        stepval = optimizer.state[optimizer.param_groups[0]["params"][-1]]
    except NameError:
        stepval=0
    return stepval

class Summaries(gym.Wrapper):
    """ Writes env summaries."""
    def __init__(self, env, prefix=None, running_mean_size=100, step_var=None):
        super(Summaries, self).__init__(env)
        self.episode_counter = 0
        self.prefix = prefix or self.env.spec.id
        self.step_count =0
        self.step_var = (step_var if step_var is not None
                         else torch_steps())

        nenvs = getattr(self.env.unwrapped, "nenvs", 1)
        self.rewards = np.zeros(nenvs)
        self.had_ended_episodes = np.zeros(nenvs, dtype=np.bool)
        self.episode_lengths = np.zeros(nenvs)
        self.reward_queues = [deque([], maxlen=running_mean_size)
                              for _ in range(nenvs)]
        self.rewards_step =0

    def should_write_summaries(self):
        """ Returns true if it's time to write summaries. """
        return np.all(self.had_ended_episodes)

    def add_summaries(self):
        """ Writes summaries. """
        print(f"Step={self.step_count}, rewards={self.rewards_step}")
        
        if 'step' not in torch_steps().keys():
            self.step_var =0
        else:
            self.step_var = torch_steps()["step"]
            
        writer.add_scalar(
            f"{self.prefix}/total_reward",
            np.mean([q[-1] for q in self.reward_queues]),
            global_step=self.step_var)
        writer.add_scalar(
            f"{self.prefix}/reward_mean_{self.reward_queues[0].maxlen}",
            np.mean([np.mean(q) for q in self.reward_queues]),
            global_step=self.step_var)
        '''
        writer.add_scalar(
            f"{self.prefix}/rewards_update",
            self.rewards_step,
            global_step=self.step_var)
        
        writer.add_scalar(
            f"{self.prefix}/episode_length",
            np.mean(self.episode_lengths),
            global_step=self.step_var)
        '''
        if self.had_ended_episodes.size > 1:
            '''
            writer.add_scalar(
                f"{self.prefix}/min_reward",
                min(q[-1] for q in self.reward_queues),
                global_step=self.step_var)
            writer.add_scalar(
                f"{self.prefix}/max_reward",
                max(q[-1] for q in self.reward_queues),
                global_step=self.step_var)
            '''
        self.episode_lengths.fill(0)
        self.had_ended_episodes.fill(False)

    def step(self, action):
        
        self.step_count +=1
        clipped_action = np.clip(action.tolist(), self.env.action_space.low, self.env.action_space.high)
        obs, rew, done, info = self.env.step(clipped_action)
        self.rewards += rew
        self.rewards_step += rew
        self.episode_lengths[~self.had_ended_episodes] += 1

        info_collection = [info] if isinstance(info, dict) else info
        done_collection = [done] if isinstance(done, bool) else done
        done_indices = [i for i, info in enumerate(info_collection)
                        if info.get("real_done", done_collection[i])]
        
        for i in done_indices:
            if not self.had_ended_episodes[i]:
                self.had_ended_episodes[i] = True
            self.reward_queues[i].append(self.rewards[i])
            self.rewards[i] = 0

        if self.should_write_summaries():
            self.add_summaries()
        return obs, rew, done, info

    def reset(self, **kwargs):
        self.rewards_step =0
        self.rewards.fill(0)
        self.episode_lengths.fill(0)
        self.had_ended_episodes.fill(False)
        return self.env.reset(**kwargs)


The normalization wrapper will subtract running mean from observations and rewards and divide 
the resulting quantities by the  running variances.

In [31]:
from mujoco_wrappers import Normalize

env = Normalize(Summaries(gym.make("HalfCheetahBulletEnv-v0")));
env.unwrapped.seed(0);

Next, you will need to define a model for training. We suggest that you use two separate networks: one for policy
and another for value function. Each network should be a 3-layer MLP with 64 hidden units, $\mathrm{tanh}$ 
activation function, kernel matrices initialized with orthogonal initializer with parameter $\sqrt{2}$
and biases initialized with zeros. 

Our policy distribution is going to be multivariate normal with diagonal covariance. 
The network from above will predict the mean, and the covariance should be represented by a single 
(learned) vector of size 6 (corresponding to the dimensionality of the action space from above). 
You should initialize this vector to zero and take the exponent of it to always
have a non-negative quantity. 

Overall the model should return three things: predicted mean of the distribution, variance vector, 
value function. 

In [32]:
# import tensorflow as tf
# import torch
import torch.nn as nn

hidden_units =64
input_shape = np.array(env.observation_space.shape).prod()
n_actions = np.prod(env.action_space.shape)


def layer_init(layer, std=np.sqrt(2), bias_const=0.0):
    # https://github.com/vwxyzjn/cleanrl/blob/418bfc01fe69712c5b617d49d810a1df7f4f0c14/cleanrl/ppo_continuous_action.py#L221
    nn.init.orthogonal_(layer.weight, std)
    nn.init.constant_(layer.bias, bias_const)
    return layer

class Model(nn.Module):
    def __init__(self,num_inputs,num_outputs):
        super(Model,self).__init__()
        self.fc1 = layer_init(nn.Linear(input_shape,64))
        self.fc2 = layer_init(nn.Linear(64,64))
        self.fc_Policy = layer_init(nn.Linear(64,n_actions))
        self.fc_Value = layer_init(nn.Linear(64,1))
        self.covariance = nn.Parameter(torch.zeros(1,n_actions))
        
    def Policy_network(self,x):
        '''
        The network predicts the mean and covariance(log of standard deviation)
        '''
        x = torch.tanh(self.fc1(x))
        x = torch.tanh(self.fc2(x))
        mean = self.fc_Policy(x)

        logstd = self.covariance.expand_as(mean)
        return mean,logstd
    
    def Value_network(self,x):
        '''
        The network predicts the Value Function
        '''
        x = torch.tanh(self.fc1(x))
        x = torch.tanh(self.fc2(x))
        x = self.fc_Value(x)
        return x


This model will be wrapped by a `Policy`. The policy can work in two modes, but in either case 
it is going to return dictionary with string-type keys. The first mode is when the policy is 
used to sample actions for a trajectory which will later be used for training. In this case 
the flag `training` passed to `act` method is `False` and the method should return 
a `dict` with the following keys: 

* `"actions"`: actions to pass to the environment
* `"log_probs"`: log-probabilities of sampled actions
* `"values"`: value function $V^\pi(s)$ predictions.

We don't need to use the values under these keys for training, so all of them should be of type `np.ndarray`.

When `training` is `True`, the model is training on a given batch of observations. In this
case it should return a `dict` with the following keys

* `"distribution"`: an instance of multivariate normal distribution (`torch.distributions.MultivariateNormal` or `tf.distributions.MultivariateNormalDiag`)
* `"values"`: value function $V^\pi(s)$ prediction.

The distinction about the modes comes into play depending on where the policy is used: if it is called from `EnvRunner`, 
the `training` flag is `False`, if it is called from `PPO`, the `training` flag is `True`. These classed 
will be described below. 

In [33]:
from torch.distributions.normal import Normal
class Policy:
  def __init__(self, model):
    self.model = model
        
    
  def act(self, inputs, training=False):
    x = torch.Tensor(inputs)
    if not training:
        '''
        training=False -- Value
        Input is Observation 
        Sample action for a Trajectory
        return {"actions:","log_probs","values"}
        '''
        with torch.no_grad():
            x = torch.Tensor(inputs).unsqueeze(0)
            mean,logstd = self.model.Policy_network(x)
            std = torch.exp(logstd)
            distrib = Normal(mean,std)
            action = distrib.sample()[0]
            log_prob = distrib.log_prob(action).sum(1).view(-1).cpu().detach().numpy()
            value = self.model.Value_network(x).view(-1).cpu().detach().numpy()
        return {"actions":action.detach().numpy(),"log_probs":log_prob,"values":value}
    else: 
        '''
        training=True - - Policy & Value
        
        Input is Observations
        return {"distribution","values"}
        '''
        x = torch.Tensor(inputs)
        mean,logstd = self.model.Policy_network(x)
        std = torch.exp(logstd)
        distrib = Normal(mean,std)
        value = self.model.Value_network(x)
        return {"distribution":distrib,"values":value}
        

We will use `EnvRunner` to perform interactions with an environment with a policy for a fixed number of timesteps. Calling `.get_next()` on a runner will return a trajectory &mdash; dictionary 
containing keys

* `"observations"`
* `"rewards"` 
* `"resets"`
* `"actions"`
* all other keys that you defined in `Policy`,

under each of these keys there is a `np.ndarray` of specified length $T$ &mdash; the size of partial trajectory. 

Additionally, before returning a trajectory this runner can apply a list of transformations. 
Each transformation is simply a callable that should modify passed trajectory in-place.

In [34]:
class AsArray:
  """ 
  Converts lists of interactions to ndarray.
  """
  def __call__(self, trajectory):
    # Modify trajectory inplace. 
    for k, v in filter(lambda kv: kv[0] != "state",
                       trajectory.items()):
      trajectory[k] = np.asarray(v)

In [35]:
import numpy as np
from runners import EnvRunner

class DummyPolicy:
  def act(self, inputs, training=False):
    assert not training
    return {"actions": np.random.randn(6), "values": np.nan}
  
runner = EnvRunner(env, DummyPolicy(), 3,
                   transforms=[AsArray()])
trajectory = runner.get_next()

{k: v.shape for k, v in trajectory.items() if k != "state"}

{'actions': (3, 6),
 'values': (3,),
 'observations': (3, 26),
 'rewards': (3,),
 'resets': (3,)}

You will need to implement the following two transformations. 

The first is `GAE` that implements [Generalized Advantage Estimator](https://arxiv.org/abs/1506.02438).
In it you should add two keys to the trajectory: `"advantages"` and `"value_targets"`. In GAE the advantages
$A_t^{\mathrm{GAE}(\gamma,\lambda)}$ are essentially defined as the exponential 
moving average with parameter $\lambda$ of the regular advantages 
$\hat{A}^{(n)}(s_t) = \sum_{l=0}^{T-1} \gamma^l r_{t+l} + \gamma^{T} V^\pi(s_{t+l}) - V^\pi(s_t)$. 
The exact formula for the computation is the following

$$
A_t^{\mathrm{GAE}(\gamma,\lambda)} = \sum_{l=0}^{T-1} (\gamma\lambda)^l\delta_{t + l}^V,
$$
where $\delta_{t+l}^V = r_{t+l} + \gamma V^\pi(s_{t+l+1}) - V^\pi(s_{t+l})$. You can look at the 
derivation (formulas 11-16) in the paper. Don't forget to reset the summation on terminal
states as determined by the flags `trajectory["resets"]`. You can use `trajectory["values"]`
to get values of all observations except the most recent which is stored under 
 `trajectory["state"]["latest_observation"]`. For this observation you will need to call the policy 
 to get the value prediction.

Once you computed the advantages, you can get the targets for training the value function by adding 
back values:
$$
\hat{V}(s_{t+l}) = A_{t+l}^{\mathrm{GAE}(\gamma,\lambda)} + V(s_{t + l}),
$$
where $\hat{V}$ is a tensor of value targets that are used to train the value function. 

In [36]:
class GAE:
  """ Generalized Advantage Estimator. """
  def __init__(self, policy, gamma=0.99, lambda_=0.95):
    self.policy = policy
    self.gamma = gamma
    self.lambda_ = lambda_
    
  def __call__(self, trajectory):
    advantages = []
    returns =[]
    lastgae = 0
    rewards = trajectory["rewards"]
    values = trajectory["values"]
    dones = 1- trajectory["resets"]
    
    #Get the latest state
    last_state = trajectory["state"]["latest_observation"]
    # Output of the network for the 'next_state' input
    network_output  =self.policy.act(last_state, training=False)
    last_value = network_output["values"]
    values = np.append(values,[last_value])# Append the next  value
    
   #https://github.com/colinskow/move37/blob/f57afca9d15ce0233b27b2b0d6508b99b46d4c7f/ppo/ppo_train.py#L69
    for step in reversed(range(len(rewards))):            
        td_delta = rewards[step] + self.gamma * values[step+1] * dones[step] - values[step]
        advantage =lastgae= td_delta + self.gamma*self.lambda_*dones[step]*lastgae
        advantages.insert(0,advantage)
        returns.insert(0,advantage+values[step])

    trajectory ["advantages"] = advantages
    trajectory ["value_targets"] = returns   
    

The main advantage of PPO over simpler policy based methods like A2C is that it is possible
to train on the same trajectory for multiple gradient steps. The following class wraps 
an `EnvRunner`. It should call the runner to get a trajectory, then return minibatches 
from it for a number of epochs, shuffling the data before each epoch.

In [37]:
class TrajectorySampler:
  """ Samples minibatches from trajectory for a number of epochs. """
  def __init__(self, runner, num_epochs, num_minibatches, transforms=None):
    self.runner = runner
    self.num_epochs = num_epochs
    self.num_minibatches = num_minibatches
    self.transforms = transforms or []
    self.minibatch_count = 0
    self.epoch_count = 0
    self.trajectory = None
    self.minibatchsize = self.num_epochs // self.num_minibatches
    self.indices = []
    
  def shuffle_trajectory(self):
    """ Shuffles all elements in trajectory.
    
    Should be called at the beginning of each epoch.
    """
    num_elements = len(self.trajectory["values"])
    self.indices  = np.arange(num_elements)
    np.random.shuffle(self.indices)
    for k,v in self.trajectory.items():
        if k != "state" :
            self.trajectory[k] = np.array(v)[self.indices.astype(int)]
    
  def get_next(self):
    """ Returns next minibatch.  """
    trajectory_minibatch ={}

    if self.epoch_count > self.num_epochs:
        return trajectory_minibatch

    if self.epoch_count ==0 and self.minibatch_count == 0:
        '''Initial Start '''
        self.trajectory = self.runner.get_next()
        self.shuffle_trajectory()
        start = 0
        end = start + self.minibatchsize-1       
        
    elif self.minibatch_count == self.num_minibatches:
        '''New Epoch'''
        self.minibatch_count=0
        self.epoch_count += 1
        self.trajectory = self.runner.get_next()
        self.shuffle_trajectory()
        start = 0
        end = start + self.minibatchsize-1
    else:
        '''Batches in the same Epoch '''
        start = self.minibatch_count*self.minibatchsize
        end = start + self.minibatchsize-1
    
    for k,v in self.trajectory.items():
        if k != "state": 
            trajectory_minibatch[k] = v[self.indices [start:end]]
    
    self.minibatch_count+=1
    
    return trajectory_minibatch
        
    

A common trick to use with GAE is to normalize advantages, the following transformation does that. 

In [38]:
class NormalizeAdvantages:
  """ Normalizes advantages to have zero mean and variance 1. """
  def __call__(self, trajectory):
    adv = trajectory["advantages"]
    adv = (adv - adv.mean()) / (adv.std() + 1e-8)
    trajectory["advantages"] = adv

Finally, we can create our PPO runner. 

In [39]:
def make_ppo_runner(env, policy, num_runner_steps=2048,
                    gamma=0.99, lambda_=0.95, 
                    num_epochs=10, num_minibatches=32):
  """ Creates runner for PPO algorithm. """
  runner_transforms = [AsArray(),
                       GAE(policy, gamma=gamma, lambda_=lambda_)]
  runner = EnvRunner(env, policy, num_runner_steps, 
                     transforms=runner_transforms)
  
  sampler_transforms = [NormalizeAdvantages()]
  sampler = TrajectorySampler(runner, num_epochs=num_epochs, 
                              num_minibatches=num_minibatches,
                              transforms=sampler_transforms)
  return sampler

In the next cell you will need to implement Proximal Policy Optimization algorithm itself. The algorithm
modifies the typical policy gradient loss in the following way:

$$
L_{\pi} = \frac{1}{T-1}\sum_{l=0}^{T-1}
\frac{\pi_\theta(a_{t+l}|s_{t+l})}{\pi_\theta^{\text{old}}(a_{t+l}|s_{t+l})}
A^{\mathrm{GAE}(\gamma,\lambda)}_{t+l}\\
L_{\pi}^{\text{clipped}} = \frac{1}{T-1}\sum_{l=0}^{T-1}\mathrm{clip}\left(
\frac{\pi_\theta(a_{t+l}|s_{t+l})}{\pi_{\theta^{\text{old}}}(a_{t+l}|s_{t+l})}
\cdot A^{\mathrm{GAE(\gamma, \lambda)}}_{t+l},
1 - \text{cliprange}, 1 + \text{cliprange}\right)\\
L_{\text{policy}} = \max\left(L_\pi, L_{\pi}^{\text{clipped}}\right).
$$

Additionally, the value loss is modified in the following way:

$$
L_V = \frac{1}{T-1}\sum_{l=0}^{T-1}(V_\theta(s_{t+l}) - \hat{V}(s_{t+l}))^2\\
L_{V}^{\text{clipped}} = \frac{1}{T-1}\sum_{l=0}^{T-1}
V_{\theta^{\text{old}}}(s_{t+l}) +
\text{clip}\left(
V_\theta(s_{t+l}) - V_{\theta^\text{old}}(s_{t+l}),
-\text{cliprange}, \text{cliprange}
\right)\\
L_{\text{value}} = \max\left(L_V, L_V^{\text{clipped}}\right).
$$

In [40]:
class PPO:
  def __init__(self, policy, optimizer,
               cliprange=0.2,
               value_loss_coef=0.25,
               max_grad_norm=0.5):
    self.policy = policy
    self.optimizer = optimizer
    self.cliprange = cliprange
    self.value_loss_coef = value_loss_coef
    # Note that we don't need entropy regularization for this env.
    self.max_grad_norm = max_grad_norm
    self.Policy_loss = 0
    self.Value_loss = 0
    self.Total_loss =0
    
  def policy_loss(self, trajectory, act):
    """ Computes and returns policy loss on a given trajectory. """
    
    actions = torch.tensor(trajectory["actions"]).to(device) 
    old_log_probs = torch.tensor(trajectory["log_probs"]).to(device).flatten() # datatype - numpy array
    new_distrib = act["distribution"] 
    new_logprobs = new_distrib.log_prob(actions).sum(1)

    
    ratio = torch.exp(new_logprobs - old_log_probs)
    surrogate1 = ratio * -torch.Tensor(trajectory["advantages"]).to(device)
    #surrogate2 = ratio.clamp(1-self.cliprange,1+self.cliprange)*-torch.Tensor(trajectory["advantages"])
    surrogate2 = torch.clamp(ratio,1-self.cliprange,1+self.cliprange)*-torch.Tensor(trajectory["advantages"])
    policy_loss = torch.mean(torch.max(surrogate1,surrogate2))
    self.Policy_loss = policy_loss
    return policy_loss
        
      
  def value_loss(self, trajectory, act):
    """ Computes and returns value loss on a given trajectory. """
    new_values = act["values"].flatten() 
    returns = torch.tensor(trajectory["value_targets"]).to(device)
    values = torch.tensor(trajectory["values"]).to(device).flatten()

    v_loss1  =(returns - new_values ).pow(2)
    clipped_values = values + torch.clamp(new_values - values,-self.cliprange,self.cliprange)
    v_loss2 = (clipped_values - returns ).pow(2)
    
    value_loss = 0.5*(torch.max(v_loss1,v_loss2)).mean()
    self.Value_loss = value_loss
    return value_loss

  def loss(self, trajectory):
    act = self.policy.act(trajectory["observations"], training=True)
    policy_loss = self.policy_loss(trajectory, act)
    value_loss = self.value_loss(trajectory, act)
    total_loss = policy_loss + self.value_loss_coef * value_loss
    self.Total_loss = total_loss # just for monitoring
    return total_loss
      
  def step(self, trajectory):
    """ Computes the loss function and performs a single gradient step. """
    #print(trajectory.keys())
    loss = self.loss(trajectory)
    optimizer.zero_grad()
    loss.backward()
    nn.utils.clip_grad_norm(self.policy.model.parameters(),self.max_grad_norm)
    optimizer.step()

Now everything is ready to do training. In one million of interactions it should be possible to 
achieve the total raw reward of about 1500. You should plot this quantity with respect to 
`runner.step_var` &mdash; the number of interactions with the environment. It is highly 
encouraged to also provide plots of the following quantities (these are useful for debugging as well):

* [Coefficient of Determination](https://en.wikipedia.org/wiki/Coefficient_of_determination) between 
value targets and value predictions
* Entropy of the policy $\pi$
* Value loss
* Policy loss
* Value targets
* Value predictions
* Gradient norm
* Advantages

For optimization it is suggested to use Adam optimizer with linearly annealing learning rate 
from 3e-4 to 0 and epsilon 1e-5.

In [41]:
import time
from gym.wrappers import  Monitor

In [42]:
from mujoco_wrappers import Normalize

suffix = f"ppo_notebook_{int(time.time())}"
writer = SummaryWriter(f"runs/{suffix}")

env = Normalize(Summaries(gym.make("HalfCheetahBulletEnv-v0")));
env.unwrapped.seed(0);

experiment_name = env.spec.id + suffix
#env = Monitor(env, f'videos/{experiment_name}')

# Enableing thrws error in Jupyter notebbok .But works in the scriot.
# Video recored with the .py file (same code)


In [43]:
# Parameters:
hidden_units =64
input_shape = np.array(env.observation_space.shape).prod()
n_actions = np.prod(env.action_space.shape)

# Linear Anneling Helper
lr = lambda f: f * learning_rate
anneal_lr = False

#PPO Hyper Params:
learning_rate = 3e-4
cliprange=0.2
value_loss_coef=0.25
max_grad_norm=0.5

##Training

'''
nsteps:number of steps of the  environment per update
total_timesteps:number of timesteps (i.e. number of actions taken in the environment)
noptepochs: number of training epochs per update
nminibatches: number of training minibatches per update

'''
nenvs =1
nsteps = num_runner_steps=2048 
num_batch = nsteps * nenvs

gamma=0.99
lambda_=0.95
num_epochs=100
num_minibatches=32

total_timesteps = 10000

#https://github.com/openai/baselines/blob/
#ea25b9e8b234e6ee1bca43083f8f3cf974143998/baselines/ppo2/ppo2.py#L99
train_mini_batch = num_batch//num_minibatches
num_updates = total_timesteps//num_minibatches

print(f"TotalSteps={total_timesteps},minibatchsize = {train_mini_batch},numUpdates={num_updates}")

TotalSteps=40000,minibatchsize = 64,numUpdates=1250


In [44]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ppo_agent = Model(input_shape,n_actions).to(device)
policy = Policy(ppo_agent)
optimizer = torch.optim.Adam(policy.model.parameters(),lr= learning_rate,eps= 1e-5 )
ppo = PPO(policy, optimizer,cliprange=0.2,value_loss_coef=0.25,max_grad_norm=0.5)


minibatcher = make_ppo_runner(env, policy, num_runner_steps,
                    gamma, lambda_, 
                    num_epochs, num_minibatches)

minibatcher.minibatchsize = train_mini_batch



In [45]:
# Training
for update in range(1,num_updates+1):
    # Linear Annealing
    minibatcher.epoch_count=0
    minibatcher.minibatch_count =0
    total_loss =[]
    value_loss =[]
    policy_loss =[]
    if anneal_lr:
        frac = 1.0 - (update - 1.0) / num_updates
        lrnow = lr(frac)
        ppo.optimizer.param_groups[0]['lr'] = lrnow
    for _ in range (num_epochs):
        for _ in range(num_minibatches):
            trajectory_mbatch = minibatcher.get_next()
            ppo.step(trajectory_mbatch)
            step = optimizer.state[optimizer.param_groups[0]["params"][-1]]["step"]

            total_loss.append(ppo.Total_loss.detach().numpy())
            value_loss.append(ppo.Value_loss.detach().numpy())
            policy_loss.append(ppo.Policy_loss.detach().numpy())
        
        writer.add_scalar(
            f"{env.spec.id}/total_loss",
            np.mean(total_loss),
            global_step=step)
        writer.add_scalar(
            f"{env.spec.id}/value_loss",
            np.mean(value_loss),
            global_step=step)
        writer.add_scalar(
            f"{env.spec.id}/policy_loss",
            np.mean(policy_loss),
            global_step=step)
    

Step=1000, rewards=-1449.6471991975902
Step=2000, rewards=-1584.3469496983053


C:\Users\NijuMalar\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:62: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.


Step=3000, rewards=-1188.830579690747
Step=4000, rewards=-1375.0890424226423
Step=5000, rewards=-1623.277119108456
Step=6000, rewards=-1553.380158702709
Step=7000, rewards=-1395.2868026014485
Step=8000, rewards=-1353.5120846179323
Step=9000, rewards=-1421.1730971309262
Step=10000, rewards=-1715.798400365278
Step=11000, rewards=-1649.3676157167752
Step=12000, rewards=-1314.4896851017681
Step=13000, rewards=-1436.934075716229
Step=14000, rewards=-1543.8626415501249
Step=15000, rewards=-1403.8575652899056
Step=16000, rewards=-1350.909989023313
Step=17000, rewards=-1492.1348738716024
Step=18000, rewards=-1485.9582353110143
Step=19000, rewards=-1526.9880541487344
Step=20000, rewards=-1640.128845749762
Step=21000, rewards=-1524.0068278302747
Step=22000, rewards=-1396.8687654773044
Step=23000, rewards=-1293.94684001562
Step=24000, rewards=-1346.5131318072506
Step=25000, rewards=-1280.6993186267039
Step=26000, rewards=-1410.3895765411096
Step=27000, rewards=-1304.744916215505
Step=28000, rewar

Step=209000, rewards=447.044709271437
Step=210000, rewards=481.89511240769957
Step=211000, rewards=410.0589119034106
Step=212000, rewards=476.2706404883315
Step=213000, rewards=470.7973033473839
Step=214000, rewards=464.41251711813544
Step=215000, rewards=427.58089198666136
Step=216000, rewards=472.4329378268423
Step=217000, rewards=471.2798007033295
Step=218000, rewards=458.5038634517065
Step=219000, rewards=464.1641169233499
Step=220000, rewards=491.3847789926069
Step=221000, rewards=484.09452554466304
Step=222000, rewards=498.41728756949794
Step=223000, rewards=438.0637326350283
Step=224000, rewards=339.38303242734514
Step=225000, rewards=487.4954208522535
Step=226000, rewards=332.5232515511285
Step=227000, rewards=478.2915142296385
Step=228000, rewards=407.6525456734079
Step=229000, rewards=455.1861685829014
Step=230000, rewards=354.8040898049377
Step=231000, rewards=303.1811711071609
Step=232000, rewards=459.7046413738642
Step=233000, rewards=455.0215673217588
Step=234000, rewards

Step=418000, rewards=528.804066013394
Step=419000, rewards=545.0402882825388
Step=420000, rewards=547.7290377734781
Step=421000, rewards=545.1163677812898
Step=422000, rewards=546.8050928057693
Step=423000, rewards=536.2373014859959
Step=424000, rewards=-71.35208624356373
Step=425000, rewards=537.6420741774473
Step=426000, rewards=553.2659931690099
Step=427000, rewards=531.1398540745867
Step=428000, rewards=547.5027304199681
Step=429000, rewards=518.2640459734245
Step=430000, rewards=532.5388427673095
Step=431000, rewards=538.4126136210847
Step=432000, rewards=548.5788398147604
Step=433000, rewards=533.068359372254
Step=434000, rewards=562.1412831817898
Step=435000, rewards=520.8617179980712
Step=436000, rewards=530.8470670242473
Step=437000, rewards=416.82945162987176
Step=438000, rewards=585.9309281883048
Step=439000, rewards=506.4144605485316
Step=440000, rewards=540.6113694301775
Step=441000, rewards=545.0969439471594
Step=442000, rewards=529.7109889211007
Step=443000, rewards=536.

...

Step=30069000, rewards=2008.055442182051
Step=30070000, rewards=1949.0432348655702
Step=30071000, rewards=1999.209712666012
Step=30072000, rewards=1883.9242438745266
Step=30073000, rewards=1976.7988913569802
Step=30074000, rewards=1844.5144921863086
Step=30075000, rewards=1909.6578867665662
Step=30076000, rewards=1944.0800027566615
Step=30077000, rewards=1953.4930141109432
Step=30078000, rewards=1988.10823303288
Step=30079000, rewards=1885.5095845539565
Step=30080000, rewards=2091.392180621149
Step=30081000, rewards=1948.8888844089388
Step=30082000, rewards=1896.9041200358608
Step=30083000, rewards=2028.7911705821
Step=30084000, rewards=1921.9023326430072
Step=30085000, rewards=1925.2651301605881
Step=30086000, rewards=1958.4185104693988
Step=30087000, rewards=1900.2397972607578
Step=30088000, rewards=2016.8799279008124
Step=30089000, rewards=1933.238567281592
Step=30090000, rewards=2006.5344276831813
Step=30091000, rewards=1957.7084371918172
Step=30092000, rewards=1956.938943685227
St

Step=30265000, rewards=1769.3998274142311
Step=30266000, rewards=1934.7577485634931
Step=30267000, rewards=1901.6633096176613
Step=30268000, rewards=1914.558275446389
Step=30269000, rewards=1885.4329064865408
Step=30270000, rewards=1825.3023215971796
Step=30271000, rewards=1897.9331063996335
Step=30272000, rewards=1941.0427159635199
Step=30273000, rewards=1860.571486724278
Step=30274000, rewards=1909.5815043418118
Step=30275000, rewards=1729.3181332528588
Step=30276000, rewards=1797.7944044756237
Step=30277000, rewards=1861.3101059423302
Step=30278000, rewards=1853.8760127451922
Step=30279000, rewards=1911.2870129336757
Step=30280000, rewards=1845.3044109745597
Step=30281000, rewards=1867.3479715142512
Step=30282000, rewards=1874.447084233248
Step=30283000, rewards=1832.3773721760183
Step=30284000, rewards=1866.1408463946989
Step=30285000, rewards=1736.6075800058309
Step=30286000, rewards=1889.708072221528
Step=30287000, rewards=1885.2934341877606
Step=30288000, rewards=1800.1872531832

Step=30461000, rewards=1938.4550947726773
Step=30462000, rewards=1886.7575971747235
Step=30463000, rewards=1887.433886908481
Step=30464000, rewards=1913.7456920520863
Step=30465000, rewards=1926.2934813122133
Step=30466000, rewards=1756.57561519491
Step=30467000, rewards=1856.355372886513
Step=30468000, rewards=1840.9381526398079
Step=30469000, rewards=1893.5601841624264
Step=30470000, rewards=1861.7690245090805
Step=30471000, rewards=1870.6938028394336
Step=30472000, rewards=1808.9460659649314
Step=30473000, rewards=1945.776557465061
Step=30474000, rewards=1896.684185417747
Step=30475000, rewards=1931.6996104300626
Step=30476000, rewards=1904.7610955703062
Step=30477000, rewards=1928.233192668377
Step=30478000, rewards=1869.2312466448202
Step=30479000, rewards=1898.38839164775
Step=30480000, rewards=1972.5271067032954
Step=30481000, rewards=1907.3500178788656
Step=30482000, rewards=1866.926461228204
Step=30483000, rewards=1864.2450040956066
Step=30484000, rewards=1923.5943300669178
St

Step=30657000, rewards=1937.2571335040363
Step=30658000, rewards=1786.4319455348507
Step=30659000, rewards=1970.2141487207816
Step=30660000, rewards=1955.367566356524
Step=30661000, rewards=1895.5387408959846
Step=30662000, rewards=1873.9522564371616
Step=30663000, rewards=1945.9888885630073
Step=30664000, rewards=1916.8838810694097
Step=30665000, rewards=1957.1477800978391
Step=30666000, rewards=1912.2653084321532
Step=30667000, rewards=2011.7362649950642
Step=30668000, rewards=1981.3032620055833
Step=30669000, rewards=1962.601678626908
Step=30670000, rewards=1855.0080845365867
Step=30671000, rewards=1925.2734373933242
Step=30672000, rewards=1952.0359681028774
Step=30673000, rewards=1986.2109811909575
Step=30674000, rewards=1909.137692831546
Step=30675000, rewards=1912.3251103877817
Step=30676000, rewards=1877.9366618864583
Step=30677000, rewards=1973.477125379057
Step=30678000, rewards=1982.2959921346217
Step=30679000, rewards=1911.1427488552656
Step=30680000, rewards=2019.7349108436

KeyboardInterrupt: 

### Tensorboard Monitor

In [20]:
%load_ext tensorboard
%tensorboard --logdir runs

Reusing TensorBoard on port 6006 (pid 12212), started 6:50:27 ago. (Use '!kill 12212' to kill it.)

### Tensorboard logs snap:
After around 8 hours.
![img](./runs/tensorboardsnapshot.JPG)

### Video
The video with running as python fine

In [27]:
# show video
from IPython.display import HTML
import os

video_names = list(
    filter(lambda s: s.endswith(".mp4"), os.listdir("./videos/")))
HTML("""
<video width="640" height="480" controls>
  <source src="{}" type="video/mp4">
</video>
""".format("./videos/"+video_names[-1]))  # this may or may not be _last_ video. Try other indices

In [28]:
env.close()